In [80]:
import requests, sys
import warnings

In [101]:
class DataCollecter():
    def __init__(self):
        self.samples = list()
        self.populations = list()
        self.population_info = dict()
        self.samples_info = dict()
        self.map_samples_to_pop = dict()
        self.map_pop_to_samples = dict()

    """ This method will return all populations of ensembl with the filter set to LD -> 1000 Genome phase 3"""
    def get_populations(self):
        if len(self.populations) == 0:
            server = "https://grch37.rest.ensembl.org"
            ext = "/info/variation/populations/homo_sapiens?filter=LD"
            r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
            if not r.ok:
                r.raise_for_status()
                sys.exit()

            populations = r.json()
            for entry in populations:
                key = entry["name"].split(":")[-1]
                self.populations.append(key)
                self.population_info[key] = entry

        return self.populations

    """ This method will return all samples from 1000GENOMES:phase_3 for all populations """
    def get_samples(self):
        if len(self.populations) == 0:
            server = "https://grch37.rest.ensembl.org"
            ext = "/info/variation/populations/human/1000GENOMES:phase_3:ALL?"

            r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

            if not r.ok:
                r.raise_for_status()
                sys.exit()

            samples = r.json()[0]["individuals"]
            for entry in samples:
                key = entry["name"].split(":")[-1]
                self.samples.append(key)
                self.samples_info[key] = entry 
        return self.samples
    
    """ Get samples by population """
    def get_samples_by_pop(self, population):
        if len(self.samples) == 0:
            self.get_samples()

        if len(self.populations) == 0:
            self.get_populations()

        if len(self.map_pop_to_samples) == 0:
            for pop in self.populations:
                server = "https://grch37.rest.ensembl.org"
                ext = "/info/variation/populations/human/1000GENOMES:phase_3:" + pop + "?"
 
                r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
                if not r.ok:
                    r.raise_for_status()
                    sys.exit()
 
                samples_pop = r.json()[0]["individuals"]
                
                for entry in samples_pop:
                    key = entry["name"].split(":")[-1]
                    self.map_samples_to_pop[key] = pop
                    item = self.map_pop_to_samples.get(pop, [])
                    item.append(key)
                    self.map_pop_to_samples[pop] = item

        if population in self.map_pop_to_samples.keys():
            return self.map_pop_to_samples[population]
        else:
            warnings.warn("This population does not exist, returned all samples")
            return self.samples
    """ This method will return the population of a sample """
    def get_pop_by_sample(self, sample):
        if len(self.map_samples_to_pop) == 0:
            this.get_samples_by_pop()

        if sample in map_samples_to_pop.keys():
            return map_samples_to_pop[sample]
        else:
            warnings.warn("This sample does not exist, returned ALL")
            return "ALL"

    def _fetch_variants(start,
                    end,
                    pageSize = 10,
                    variantAnnotationSetId = 'Ensembl', 
                    reference = "a718acaa6135fdca8357d5bfe94211dd"):
        server = "https://grch37.rest.ensembl.org"
        ext = "/ga4gh/variantannotations/search"
        headers={ "Content-Type" : "application/json", "Accept" : "application/json"}
        r = requests.post(server+ext, headers=headers, data='{"pageSize": "' + pageSize +'",                "variantAnnotationSetId": "' + variantAnnotationSetId + '", "referenceId": "' + reference + '", "start": '+start+' , "end": ' + end + ' }')
 
        if not r.ok:
            r.raise_for_status()
            sys.exit()
        
        decoded = r.json()
        return decoded

        # TODO iterate over page tokens






In [79]:
data = VariationCollecter()
VariationCollecter._fetc("ALL")

KeyError: 'ALL'

In [78]:
data.get_samples_by_pop("ASW")

['NA20412',
 'NA20362',
 'NA20359',
 'NA20357',
 'NA20356',
 'NA20355',
 'NA20351',
 'NA20348',
 'NA20346',
 'NA20342',
 'NA20340',
 'NA20339',
 'NA20334',
 'NA20332',
 'NA20321',
 'NA20320',
 'NA20318',
 'NA20317',
 'NA20314',
 'NA20299',
 'NA20298',
 'NA20296',
 'NA20294',
 'NA20291',
 'NA20289',
 'NA20287',
 'NA20282',
 'NA20281',
 'NA20278',
 'NA20276',
 'NA20274',
 'NA20127',
 'NA20126',
 'NA19984',
 'NA19982',
 'NA19923',
 'NA19922',
 'NA19921',
 'NA19920',
 'NA19917',
 'NA19916',
 'NA19914',
 'NA19913',
 'NA19909',
 'NA19908',
 'NA19904',
 'NA19901',
 'NA19900',
 'NA19835',
 'NA19834',
 'NA19819',
 'NA19818',
 'NA19713',
 'NA19712',
 'NA19711',
 'NA19707',
 'NA19704',
 'NA19703',
 'NA19701',
 'NA19700',
 'NA19625']

In [15]:
server = "https://grch37.rest.ensembl.org"
ext = "/info/variation/populations/human/1000GENOMES:phase_3:ALL?"
 
r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
decoded = r.json()
decoded[0].keys()

dict_keys(['size', 'description', 'individuals', 'name'])

In [42]:
decoded[0]["individuals"][0]["name"]

'1000GENOMES:phase_3:NA21144'

In [6]:
server = "https://grch37.rest.ensembl.org"
ext = "/ga4gh/variants/search"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}
r = requests.post(server+ext, headers=headers, data='{ "variantSetId": 3, "callSetIds":[ "3:NA19777"  , "3:HG01242" , "3:HG01142"   ], "referenceName": 22,"start": 17190024 ,"end":  17671934 }')
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
decoded = r.json()
decoded.keys()

dict_keys(['nextPageToken', 'variants'])

In [7]:
decoded['variants']

[{'start': 17190036,
  'names': ['rs533198068'],
  'created': 1432745640000,
  'referenceBases': 'A',
  'referenceName': '22',
  'alternateBases': ['G'],
  'id': '3:rs533198068',
  'calls': [{'callSetId': '3:HG01142',
    'phaseset': '',
    'info': {},
    'genotypeLikelihood': [],
    'callSetName': 'HG01142',
    'genotype': [0, 0]},
   {'info': {},
    'callSetName': 'HG01242',
    'genotypeLikelihood': [],
    'genotype': [0, 0],
    'callSetId': '3:HG01242',
    'phaseset': ''},
   {'genotype': [0, 0],
    'info': {},
    'callSetName': 'NA19777',
    'genotypeLikelihood': [],
    'phaseset': '',
    'callSetId': '3:NA19777'}],
  'info': {'SAS_AF': ['0.0000'],
   'AMR_AF': ['0.0000'],
   'AF': ['0.000998403'],
   'EAS_AF': ['0.0000'],
   'ssID': ['ss1366601865'],
   'DP': ['12306'],
   'AFR_AF': ['0.0038'],
   'EUR_AF': ['0.0000'],
   'AN': ['5008'],
   'AC': ['5'],
   'NS': ['2504']},
  'end': 17190037,
  'variantSetId': '3',
  'updated': 1432745640000},
 {'referenceName': '22',

In [8]:
server = "https://grch37.rest.ensembl.org"
ext = "/ga4gh/datasets/search"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}
r = requests.post(server+ext, headers=headers, data='{ "pageSize": 3 }')
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
decoded = r.json()
print(repr(decoded))

{'datasets': [{'name': '1000 Genomes phase3', 'id': '6e340c4d1e333c7a676b1710d2e3953c', 'description': '1000 Genomes phase3 genotypes'}, {'name': 'Ensembl', 'id': 'Ensembl', 'description': 'Ensembl annotation'}, {'description': 'Illumina platinum genomes genotypes', 'name': 'Illumina platinum genomes', 'id': 'c7d2c4a0e0dcb28bdb30559f16c7819d'}], 'nextPageToken': 'da67c32cf330c0a61c014322ebc4e50a'}


In [105]:
server = "https://grch37.rest.ensembl.org"
ext = "/ga4gh/variantannotations/search"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}
r = requests.post(server+ext, headers=headers, data='{"pageSize": 1000, "variantAnnotationSetId": "Ensembl", "referenceId": "a718acaa6135fdca8357d5bfe94211dd", "start": 25617400 , "end": 25618500, "pageToken": 25617811 }')
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
decoded = r.json()
decoded.keys()

dict_keys(['variantAnnotations', 'nextPageToken'])

In [110]:
len(decoded["variantAnnotations"])

157